# Linear Regression

A notebook reproducing some of the examples from chapter 3 of Hastie, Tibshirani, Friedman, *The Elements of Statistical Learning*

In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as spl
import scipy.stats as sps
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
from mlalgos.supervised_learning.linear_regression import (
    DummyRegression,
    LinearRegression,
    LassoRegression,
    PrincipalComponentsRegression,
    RidgeRegression,
    PartialLeastSquaresRegression
)
from mlalgos.utils.data_utils import StandardScaler

## Import some data

In [3]:
prostate_data = '~/datasets/prostate.txt'
df = pd.read_csv(prostate_data, sep='\t', usecols=[1,2,3,4,5,6,7,8,9,10])
df.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [4]:
target = 'lpsa'
features = list(df)
features.remove('lpsa')
features.remove('train')
print(features)

['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']


In [5]:
# use train flag to split
train = df.train == 'T'
X, y = df[features].values, df[target].values
X_train, y_train = X[train], y[train]
X_test, y_test = X[~train], y[~train]

In [6]:
# correlation matrix
corr = []
for feature1 in features:
    corr_list = []
    x1 = df[train][feature1].values
    mean1 = np.mean(x1)
    std1 = np.std(x1)
    for feature2 in features:
        x2 = df[train][feature2].values
        mean2 = np.mean(x2)
        std2 = np.std(x2)
        c = np.mean((x1-mean1)*(x2-mean2))/(std1*std2)
        corr_list.append(c)
    corr.append(corr_list)
print(corr)

[[1.0000000000000002, 0.30023198690216274, 0.28632426557510204, 0.06316772022296163, 0.592949130529482, 0.6920430753226345, 0.4264140724486782, 0.48316135710366304], [0.30023198690216274, 0.9999999999999999, 0.31672346842086035, 0.437041536580065, 0.1810544782843504, 0.15682859478541616, 0.023558207288831432, 0.0741663208573534], [0.28632426557510204, 0.31672346842086035, 0.9999999999999999, 0.28734644573788043, 0.12890226303142774, 0.172951397595152, 0.3659151225132289, 0.2758057291244312], [0.06316772022296163, 0.437041536580065, 0.28734644573788043, 0.9999999999999998, -0.13914679926810444, -0.08853455936907369, 0.032992152046930436, -0.030403819438876666], [0.592949130529482, 0.1810544782843504, 0.12890226303142774, -0.13914679926810444, 1.0, 0.6712402103032987, 0.3068753723785833, 0.48135774093302786], [0.6920430753226345, 0.15682859478541616, 0.172951397595152, -0.08853455936907369, 0.6712402103032987, 1.0000000000000002, 0.4764368357350071, 0.6625333515651151], [0.42641407244867

In [7]:
# correlation matrix using pandas (tab. 3.1)
df[train][features].corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000


In [8]:
# base error rate by simply predicting the mean
dummy_model = DummyRegression()
dummy_model.fit(X_train, y_train)
base_error_rate = mean_squared_error(y_test, dummy_model.predict(X_test))
print('Base error rate: ', base_error_rate)

Base error rate:  1.0567332280603818


In [9]:
# standardise for unit variance
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# The difference in parameters compared to Tab 3.2 in ESL is due to standardising the data only on the training data,
# rather than the whole dataset, as done in the book. This propagates to later results
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
beta_names = ['Intercept'] + features
for i in range(len(beta_names)):
    print(beta_names[i], '\t', linear_model.beta_hat[i])

Intercept 	 2.4523450850746276
lcavol 	 0.7110405922561779
lweight 	 0.2904502919864316
age 	 -0.14148182348942576
lbph 	 0.210419510184796
svi 	 0.3073002529718587
lcp 	 -0.28684074913663926
gleason 	 -0.020756862036926726
pgg45 	 0.27526842547776587


In [16]:
# standard errors
X_tmp = np.insert(X_train, 0, 1, axis=1)
vary = np.sum((y_train - linear_model.predict(X_train))**2)/(len(y_train)-len(features)-1)
errs = np.sqrt(np.diagonal(spl.inv(X_tmp.T @ X_tmp)) * vary)
print(errs)

[0.08701959 0.13250132 0.10558798 0.10135462 0.1023518  0.12445059
 0.15364444 0.14151003 0.1583969 ]


In [19]:
# z score
z = []
for i in range(len(beta_names)):
    z.append(linear_model.beta_hat[i]/errs[i])
print(z)

[28.18152744197761, 5.3662904561505105, 2.750789389869384, -1.3959089818189574, 2.0558456259309086, 2.4692551777938414, -1.8669126353948038, -0.14668120644371685, 1.737839719569908]


In [20]:
result = zip(beta_names, linear_model.beta_hat, errs, z)
print('Term            Coeff    Std. Err      Z Score')
print('-----------------------------------------------')
for term, coefficient, std_err, z_score in result:
    print(f'{term:>10}{coefficient:>10.2f}{std_err:>9.2f}{z_score:>15.2f}')

Term            Coeff    Std. Err      Z Score
-----------------------------------------------
 Intercept      2.45     0.09          28.18
    lcavol      0.71     0.13           5.37
   lweight      0.29     0.11           2.75
       age     -0.14     0.10          -1.40
      lbph      0.21     0.10           2.06
       svi      0.31     0.12           2.47
       lcp     -0.29     0.15          -1.87
   gleason     -0.02     0.14          -0.15
     pgg45      0.28     0.16           1.74


In [21]:
# prediction error
pred_err = mean_squared_error(linear_model.predict(X_test), y_test)
print(f'{"Prediction error":20}{pred_err:5.2f}')
print(f'{"Base error":20}{base_error_rate:5.2f}')

Prediction error     0.52
Base error           1.06


In [22]:
# F score from dropping age, lcp, gleason, pgg45 (the features with lowest absolute z-score)
N = len(y_train)
RSS1 = mean_squared_error(linear_model.predict(X_train), y_train) * N
betas0 = linear_model.beta_hat.copy()
indxs = list(np.where(np.array(np.abs(z)) < 2)[0])
print('Dropping: ', np.array(beta_names)[indxs])
features0 = features.copy()
for i in indxs:
    features0.remove(beta_names[i])
X0_train = df[features0].values[train]
linear_model0 = LinearRegression()
linear_model0.fit(X0_train, y_train)
RSS0 = mean_squared_error(linear_model0.predict(X0_train), y_train) * N
p1 = 9
p0 = 5
F = ((RSS0-RSS1)/(p1-p0))/(RSS1/(N-p1))
Fdist = sps.f(p1-p0,N-p1-1)
p_val = Fdist.sf(F)
print(f'{"F = "}{F:.2f}')
print(f'{"p-value = "}{p_val:.2f}')

Dropping:  ['age' 'lcp' 'gleason' 'pgg45']
F = 1.67
p-value = 0.17


Thus we cannot reject the null hypothesis (of setting the four features with lowest absolute z-score to zero) as the p=0.05 level.